In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from keras.regularizers import l1_l2, l2
from keras.optimizers import Adam
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings('ignore')

In [518]:
df = pd.read_csv('veda_mc.csv')
df.head()

,Unnamed: 0,date,amount
0,0,"Jul 30, 2023",1.0
1,1,"Aug 01, 2023",583.0
2,2,"Aug 01, 2023",93.0
3,3,"Aug 04, 2023",454.0
4,4,"Aug 09, 2023",5500.0


In [519]:
df = df.drop(columns=['Unnamed: 0'])

In [520]:
df = df.rename(columns = {'date': 'date',
          'amount': 'price'})

In [521]:
df['date'] = pd.to_datetime(df['date'])
date_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
full_df = pd.DataFrame(date_range, columns=['date'])
df = pd.merge(full_df, df, on='date', how='left')
df['price'] = df['price'].fillna(0)

In [522]:
df = df.groupby('date').agg({
    'price': 'sum'
}).rename(columns={'price': 'total_price'})
df.reset_index(inplace=True)

In [523]:
import pandas as pd

def calculate_monthly_avg(df):
    df['date'] = pd.to_datetime(df['date'])
    df['monthly_avg'] = None
    monthly_spending = {}

    for idx, row in df.iterrows():
        current_month = row['date'].month
        current_year = row['date'].year
        month_key = (current_year, current_month)

        if month_key not in monthly_spending:
            monthly_spending[month_key] = []

        monthly_spending[month_key].append(row['total_price'])
        monthly_avg = sum(monthly_spending[month_key]) / len(monthly_spending[month_key])
        
        df.at[idx, 'monthly_avg'] = monthly_avg

calculate_monthly_avg(df)


In [524]:
def extract_date_features(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df['day_of_week'] = df[date_col].dt.dayofweek    
    return df
df = extract_date_features(df, 'date')
encoder = OneHotEncoder(sparse_output=False)
encoded_days = encoder.fit_transform(df[['day_of_week']])
encoded_df = pd.DataFrame(encoded_days, columns=[f'day_of_week_{i}' for i in range(1, 8)])
df = pd.concat([df, encoded_df], axis=1)
df = df.drop(columns=['day_of_week'])


In [525]:
lag_values = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
for lag in lag_values:
    df[f'lag_{lag}'] = sum([df['total_price'].shift(i) for i in range(1, lag+1)]) / lag

In [526]:
df.dropna(inplace=True)

In [527]:
df.sort_values('date', inplace=True)
df.set_index('date', inplace=True)

In [528]:
df['total_price'].describe()

count     573.000000
mean      710.503735
std      1277.167773
min         0.000000
25%        20.000000
50%       255.000000
75%       870.000000
max      9615.000000
Name: total_price, dtype: float64

In [529]:
old_mean = df['total_price'].mean()
old_mean

710.5037347294939

In [530]:
filtered_df = df[df['total_price'] > 0]
Q1 = filtered_df['total_price'].quantile(0.25)
Q3 = filtered_df['total_price'].quantile(0.75)
IQR = Q3-Q1
original_values = df['total_price'].copy()
df.loc[df['total_price'] > Q3 + IQR, 'total_price'] = Q3 + IQR

In [531]:
filtered_df = df[df['total_price'] > 0]

In [532]:
new_mean = df['total_price'].mean()
new_mean

537.1546945898779

In [533]:
Q1

180.0

In [534]:
Q3

1101.5

In [535]:
df['total_price'].describe()

count     573.000000
mean      537.154695
std       640.863782
min         0.000000
25%        20.000000
50%       255.000000
75%       870.000000
max      2023.000000
Name: total_price, dtype: float64

In [536]:
x = df.drop(columns=['total_price'])
y = df['total_price']
scale_x = StandardScaler()
scale_y = StandardScaler()
x_scaled = scale_x.fit_transform(x)
y_scaled = scale_y.fit_transform(y.values.reshape(-1,1))
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.25, random_state=42)
model = ElasticNet(
    alpha=0.00001,
    l1_ratio=0.75,
    max_iter=200000,
    tol=0.0001,
    fit_intercept=True,
    random_state=42
)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred_original = scale_y.inverse_transform(y_pred.reshape(-1,1))
y_test_original = scale_y.inverse_transform(y_test.reshape(-1,1))

In [537]:
mean_change = old_mean - new_mean

In [538]:
mean_change

173.34904013961602

In [539]:
l = []
from calendar import monthrange

last_date = df.index[-1]
year, month = last_date.year, last_date.month 
days_in_month = monthrange(year, month)[1]
from datetime import timedelta
for i in range(days_in_month):
    last_date = df.index[-1]
    next_day = last_date + timedelta(days=1)
    next_data = {}
    current_month = next_day.month
    monthly_data = df[df.index.month == current_month]
    next_data['monthly_avg'] = monthly_data['total_price'].mean()
    next_data['day_of_week_1'] = 1 if next_day.weekday() == 0 else 0
    next_data['day_of_week_2'] = 1 if next_day.weekday() == 1 else 0
    next_data['day_of_week_3'] = 1 if next_day.weekday() == 2 else 0
    next_data['day_of_week_4'] = 1 if next_day.weekday() == 3 else 0
    next_data['day_of_week_5'] = 1 if next_day.weekday() == 4 else 0
    next_data['day_of_week_6'] = 1 if next_day.weekday() == 5 else 0
    next_data['day_of_week_7'] = 1 if next_day.weekday() == 6 else 0
    lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
    for lag in lags:
        lag_value = df['total_price'].shift(lag).iloc[-1]
        next_data[f'lag_{lag}'] = lag_value
    next_data_df = pd.DataFrame([next_data], index=[next_day])
    next_data_scaled = scale_x.transform(next_data_df)
    predicted_value = model.predict(next_data_scaled)
    predicted_value = scale_y.inverse_transform(predicted_value.reshape(-1, 1))
    if predicted_value[0] < 10:
        predicted_value[0] = 0

    next_data_df['total_price'] = predicted_value[0]
    df = pd.concat([df, next_data_df], axis=0)
    l.append((predicted_value[0], next_day.weekday()))


In [540]:
l

[(array([975.80999392]), 5),
 (array([0.]), 6),
 (array([1524.98087422]), 0),
 (array([0.]), 1),
 (array([1259.53269703]), 2),
 (array([89.05950603]), 3),
 (array([748.29541587]), 4),
 (array([838.98620696]), 5),
 (array([0.]), 6),
 (array([650.42383226]), 0),
 (array([0.]), 1),
 (array([887.69493766]), 2),
 (array([582.60631021]), 3),
 (array([518.66803068]), 4),
 (array([1072.89634441]), 5),
 (array([0.]), 6),
 (array([348.88368212]), 0),
 (array([0.]), 1),
 (array([643.82175869]), 2),
 (array([128.75012676]), 3),
 (array([0.]), 4),
 (array([225.63411212]), 5),
 (array([0.]), 6),
 (array([696.87724164]), 0),
 (array([0.]), 1),
 (array([614.01717044]), 2),
 (array([0.]), 3),
 (array([268.37166838]), 4),
 (array([610.34582234]), 5),
 (array([79.35895131]), 6),
 (array([842.46995947]), 0)]

In [541]:
sum_ = sum([item[0] for item in l])+days_in_month*mean_change
average = sum_ / len(l) if len(l) > 0 else 0

In [542]:
sum_

array([18981.30488684])

In [543]:
average

array([612.30015764])